In [43]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [70]:
# Lessons on strings of bytes, hex and bin notation
# splitting and joining
print(list(b"\xAE\x00"))


[174, 0]


In [ ]:
# Then UART interface datasheet

In [71]:
import time
from machine import UART

uart2 = UART(2, baudrate=115200, rx=2, tx=18)


In [ ]:
# Need to make screenshots of the manual as we work through it

In [86]:
# Read the software version
uart2.write(bytes((0xAA, 0x01, 0x04, 2)))
time.sleep_ms(20)
print(uart2.read())

b'\xbb\x02\x08\x03'


In [67]:
# put into op mode
uart2.write(bytes((0xAA, 0x00, 0x3D, 1, 0x0C)))
time.sleep_ms(20)
v = uart2.read()
print(v)


b'\xee\x01'


In [87]:
# The 0x80 android notation
uart2.write(bytes((0xAA, 0x01, 0x3B, 1)))
time.sleep_ms(20)
print(uart2.read())


b'\xbb\x01\x00'


In [113]:
# Now get the pitch roll and heading
uart2.write(bytes((0xAA, 0x01, 0x1A, 6)))
time.sleep_ms(10)
v = uart2.read()
print(v)
v = v[2:]
print((v[0]+v[1]*256)/16)
print([x/16  for x in ustruct.unpack("<hhh", v)])

b'\xbb\x06\xaa\x15\t\x00\xcf\xfc'
346.625
[346.625, 0.5625, -51.0625]


In [117]:
uart2.write(bytes((0xAA, 0x01, 0x20, 22)))
time.sleep_ms(10)
v = uart2.read()
print(v)


b'\xbb\x16\x1e\x08\x8e\xffo\xff\x85\xc0\x01\x00\xfb\xff\xff\xff\x0f\x00\x0f\x00\xd4\x03\x19?'


In [119]:
print(ustruct.unpack("<hhhhhhhhhh", v[2:]))  
# See; it's about 9.8m/s^2

(2078, -114, -145, -16251, 1, -5, -1, 15, 15, 980)


In [ ]:
# UDP technology

In [123]:
# Demo of available Wifis
import network
si = network.WLAN(network.STA_IF) 
si.active(True)
k = si.scan()
print(k)

[(b'DoESLiverpool', b'\xf0\x9f\xc2\xf4\x1b\xe4', 6, -59, 3, False), (b'Find Out Digital', b'\x02\x9f\xc2\xf4\x1b\xe4', 6, -59, 3, False), (b'Find Out Digital', b'\x02\x9f\xc2\xf4\x1a\x7f', 6, -77, 3, False), (b'DoESLiverpool', b'\xf0\x9f\xc2\xf4\x1a\x7f', 6, -77, 3, False), (b'Find Out Digital', b'\x02\x9f\xc2\xf4.\xf4', 11, -80, 3, False), (b'DoESLiverpool', b'\xf0\x9f\xc2\xf4.\xf4', 11, -82, 3, False), (b'BB', b'\xf0\x9f\xc2\xdd\xd8^', 1, -89, 3, False), (b'Fabric Free Wi-Fi', b'\xf2\x9f\xc2\xdd\xd8^', 1, -90, 0, False)]


In [125]:
# Now do UDP packets 
import network, socket, time

si = network.WLAN(network.AP_IF) 
si.active(True)         
print(si.ifconfig())


('192.168.4.1', '255.255.255.0', '192.168.4.1', '0.0.0.0')


In [126]:
espname = "ESP_%X%X%X" % tuple(si.config("mac")[-3:])
print("\nConnect to wifi {}".format(espname))
port = 9019

sockudp = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sockudp.settimeout(0.1)
sockudp.bind(("192.168.4.1", port))



Connect to wifi ESP_8C1FB2


In [131]:
sockudp.settimeout(1000)
rmess, udpaddr = sockudp.recvfrom(100)
print(rmess, udpaddr)

....b'HiThere2' ('192.168.4.3', 41427)


In [132]:
for i in range(1000):
    sockudp.sendto(b"T%d"%i, udpaddr)
    time.sleep_ms(100)


.....

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
KeyboardInterrupt: 


In [155]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [156]:
%sendtofile main.py

import network, socket, time, ustruct
from machine import UART

si = network.WLAN(network.AP_IF) 
si.active(True)         
port = 9019
sockudp = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sockudp.settimeout(10000)
sockudp.bind(("192.168.4.1", port))
rmess, udpaddr = sockudp.recvfrom(100)
print("UDP socket", udpaddr)

uart2 = UART(2, baudrate=115200, rx=2, tx=18)

uart2.write(bytes((0xAA, 0x00, 0x3D, 1, 0x0C)))
time.sleep_ms(20)
print(uart2.read())

sockudp.sendto(b"n,t,qw,qx,qy,qz,lx,ly,lz,gx,gy,gz,temp,calibstat\n", udpaddr)
n = 0
while True:
    t = time.ticks_ms()
    uart2.write(bytes((0xAA, 0x01, 0x20, 24)))
    time.sleep_ms(4)
    v = uart2.read()
    if v is not None and len(v) == 26:
        qw,qx,qy,qz, lx,ly,lz, gx,gy,gz, temp,calibstat = \
            ustruct.unpack("<hhhhhhhhhhbB", v[2:])
        s = (n, t, qw,qx,qy,qz, lx/100,ly/100,lz/100, gx/100,gy/100,gz/100, temp,calibstat)
        m = b"%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%d,%d\n" % s
        sockudp.sendto(m, udpaddr)
        n += 1
    time.sleep_ms(6)


Sent 33 lines (1057 bytes) to main.py.


In [ ]:
# deal with calibration!!!